## First Layer

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1248, 768)

soups = []
for i in range(1,6):
    driver.get("https://us.shein.com/daily-new.html?ici=CCCSN%3DWomenHomePage_ON%3DBanner_OI%3D1_CN%3Ddaily_TI%3D50001_aod%3D0_PS%3D11-1_ABT%3DSPcCccWomenHomepage_default_8366&cat_ids=2030&daily=2020-12-22&page="+str(i))
    time.sleep(1.5)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    counter = 1
    while True:
        
        # Scroll down to bottom
        driver.execute_script("window.scrollBy(0, 768);")

        # Wait to load page
        time.sleep(.2)

        # Calculate new scroll height and compare with last scroll height
        new_height = 768 * counter
        if new_height >= last_height:
            break

        counter += 1

    soup = BeautifulSoup(driver.page_source,'html.parser')
    soups.append(soup)

In [2]:
product_name = []
original_price = []
sale_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []

for soup in soups:
    for i in soup.find_all('section',{'class':'S-product-item j-expose__product-item product-list__item'}):
        if i.find('img',{'class':'falcon-lazyload'}) != None:
            product_name.append(i.find('img',{'class':'falcon-lazyload'}).get('alt'))
        else:
            product_name.append(None)

        if i.find('span',{'class':'S-product-item__retail-price S-product-item__retail-price_line'}) != None:
            sale_price.append((i.find('span',{'class':'S-product-item__sale-price'})).text.strip().replace('US',''))
            original_price.append((i.find('span',{'class':'S-product-item__retail-price S-product-item__retail-price_line'})).text.strip().replace('US',''))
        elif i.find('span',{'class':'S-product-item__retail-price'}) != None:
            sale_price.append((i.find('span',{'class':'S-product-item__retail-price'})).text.strip().replace('US',''))
            original_price.append(None)
        else:
            sale_price.append(None)
            original_price.append(None)
        
        if i.find('a',{'class':'S-product-item__img-container j-expose__product-item-img'}) != None:
            product_url.append("https://us.shein.com"+i.find('a',{'class':'S-product-item__img-container j-expose__product-item-img'}).get('href'))

        if i.find('img',{'class':'falcon-lazyload'}) !=None:
            image_url.append("https:"+(i.find('img',{'class':'falcon-lazyload'})).get('data-src'))
        else:
            image_url.append(None)
        brand.append("shein")
        product_type.append("Women's Clothing")
        timestamp.append(date_today)

In [3]:
column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [4]:
df

,product_name,original_price,sale_price,product_link,image_link
0,SHEIN Strappy Side Cami Top,None,$11.00,https://us.shein.com/Strappy-Side-Cami-Top-p-1...,https://img.ltwebstatic.com/images3_pi/2020/12...
1,SHEIN One Shoulder Cut Out Front Allover Print...,None,$8.00,https://us.shein.com/One-Shoulder-Cut-Out-Fron...,https://img.ltwebstatic.com/images3_pi/2020/12...
2,SHEIN Lace Up Backless Split Thigh Side Dress,None,$13.00,https://us.shein.com/Lace-Up-Backless-Split-Th...,https://img.ltwebstatic.com/images3_pi/2020/12...
3,SHEIN Zip Half Placket Split Side Allover Prin...,None,$11.00,https://us.shein.com/Zip-Half-Placket-Split-Si...,https://img.ltwebstatic.com/images3_pi/2020/12...
4,SHEIN Cut Out Waist Halter Jumpsuit,None,$18.00,https://us.shein.com/Cut-Out-Waist-Halter-Jump...,https://img.ltwebstatic.com/images3_pi/2020/12...
...,...,...,...,...,...
595,Scoop Neck Lantern Sleeve Tee,None,$12.00,https://us.shein.com/Scoop-Neck-Lantern-Sleeve...,https://img.ltwebstatic.com/images3_pi/2020/12...
596,Tie Neck Lantern Sleeve Patchwork Top,None,$13.00,https://us.shein.com/Tie-Neck-Lantern-Sleeve-P...,https://img.ltwebstatic.com/images3_pi/2020/10...
597,Colorblock Twist Hem Waffle Knit Tee,None,$16.00,https://us.shein.com/Colorblock-Twist-Hem-Waff...,https://img.ltwebstatic.com/images3_pi/2020/10...
598,Single Breasted Tie Neck Blouse,None,$15.00,https://us.shein.com/Single-Breasted-Tie-Neck-...,https://img.ltwebstatic.com/images3_pi/2020/11...


## Second Layer

In [5]:
product_description = []
material = []
picture_name = []
color = []
counter = 1
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')

for link in product_url:
    driver.get(link)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    if soup.find('div',{'class':'product-intro__head-name'}) != None:
        color.append(None)

        if soup.find('div',{'class':'product-intro__description-panel'}) != None:
            product_description.append(soup.find('div',{'class':'product-intro__description-panel'}).text.strip())
        else:
            product_description.append(None)

        material_item = []

        if soup.find('div',{'class':'val'}) != None:
            for j in soup.find_all('div',{'class':'val'}):
                material_item.append(j.text.strip())
            material.append(material_item)
        else:
            material.append(None)
    else:
        color.append(None)
        product_description.append(None)
        material.append(None)

    picture_name.append('shein-women_' + str(counter) + '.jpg') # change variable
    counter += 1
driver.close()

In [6]:
# data cleaning for material
material_clean = []
for i,v in enumerate(material):
    if v != None:
        item_material = []
        for j in v:
            if "%" in j:
                item_material.append(j)
        material_clean.append(item_material)
    else:
        material_clean.append(None)

for i,v in enumerate(material):
    if len(v) > 3:
        color[i] = v[2]

In [13]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material_clean
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,shein,Women's Clothing,SHEIN Strappy Side Cami Top,Style: \n Sexy\n ...,None,$11.00,White,"[5% Spandex, 95% Polyester]",https://us.shein.com/Strappy-Side-Cami-Top-p-1...,https://img.ltwebstatic.com/images3_pi/2020/12...,2020-12-22,shein-women_1.jpg
1,shein,Women's Clothing,SHEIN One Shoulder Cut Out Front Allover Print...,Style: \n Casual\n ...,None,$8.00,Black and White,"[5% Spandex, 95% Polyester]",https://us.shein.com/One-Shoulder-Cut-Out-Fron...,https://img.ltwebstatic.com/images3_pi/2020/12...,2020-12-22,shein-women_2.jpg
2,shein,Women's Clothing,SHEIN Lace Up Backless Split Thigh Side Dress,Style: \n Sexy\n ...,None,$13.00,"Hot Pink, Bright","[8% Spandex, 92% Polyester]",https://us.shein.com/Lace-Up-Backless-Split-Th...,https://img.ltwebstatic.com/images3_pi/2020/12...,2020-12-22,shein-women_3.jpg
3,shein,Women's Clothing,SHEIN Zip Half Placket Split Side Allover Prin...,Style: \n Casual\n ...,None,$11.00,Black and White,"[5% Spandex, 95% Polyester]",https://us.shein.com/Zip-Half-Placket-Split-Si...,https://img.ltwebstatic.com/images3_pi/2020/12...,2020-12-22,shein-women_4.jpg
4,shein,Women's Clothing,SHEIN Cut Out Waist Halter Jumpsuit,Style: \n Sexy\n ...,None,$18.00,Black,"[13% Spandex, 87% Polyester]",https://us.shein.com/Cut-Out-Waist-Halter-Jump...,https://img.ltwebstatic.com/images3_pi/2020/12...,2020-12-22,shein-women_5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
595,shein,Women's Clothing,Scoop Neck Lantern Sleeve Tee,Style: \n Casual\n ...,None,$12.00,White,"[5% Spandex, 95% Cotton]",https://us.shein.com/Scoop-Neck-Lantern-Sleeve...,https://img.ltwebstatic.com/images3_pi/2020/12...,2020-12-22,shein-women_596.jpg
596,shein,Women's Clothing,Tie Neck Lantern Sleeve Patchwork Top,Style: \n Elegant\n ...,None,$13.00,Multicolor,[100% Polyester],https://us.shein.com/Tie-Neck-Lantern-Sleeve-P...,https://img.ltwebstatic.com/images3_pi/2020/10...,2020-12-22,shein-women_597.jpg
597,shein,Women's Clothing,Colorblock Twist Hem Waffle Knit Tee,Style: \n Casual\n ...,None,$16.00,Dusty Pink,"[65% Cotton, 35% Polyester]",https://us.shein.com/Colorblock-Twist-Hem-Waff...,https://img.ltwebstatic.com/images3_pi/2020/10...,2020-12-22,shein-women_598.jpg
598,shein,Women's Clothing,Single Breasted Tie Neck Blouse,Style: \n Elegant\n ...,None,$15.00,Dusty Pink,"[3% Spandex, 97% Polyester]",https://us.shein.com/Single-Breasted-Tie-Neck-...,https://img.ltwebstatic.com/images3_pi/2020/11...,2020-12-22,shein-women_599.jpg


In [14]:
second_pass.to_csv("shein-women.csv", index=False, encoding = 'utf-8-sig')

In [15]:
import os
import requests as req

new_pd = pd.read_csv('shein-women.csv', encoding='utf-8-sig')

os.mkdir('shein-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('shein-women/'+ "shein-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1